In [1]:
# OpenCV 한테 숫자 줌 -> 그게 패스워드가됨 -> 그 패스워드가 맞는지 아닌지?
# import cv2 as cv
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 딥러닝 모델 준비 (파일 7 내용)
def train_digital_model():
    # MNIST: 전 세계 사람들이 쓴 숫자(0~9) 이미지 6만장이 들어있는 교과서
    # MNIST 모델은 입구(입력층)가 784개(28*28)로 설계되어 있다. AI는 자기가 배운 784개의 점(28*28 사이즈) 위치에 맞춰서 숫자를 판단한다. 그래서 아무리 큰 사진을 찍어도 AI가 아는 크기인 28*28로 딱 맞춰서 건네줘야 AI가 당황하지 않고 "7이다!" 라고 대답할 수 있다
    mnist = keras.datasets.mnist
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    train_images, test_images = train_images / 255.0, test_images / 255.0

    model = keras.Sequential([
        keras.layers.Input(shape=(28, 28)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    print("모델 학습 중...")

    # model: ai의 두뇌, layer를 여러개 쌓아서 복잡한 숫자 모양을 기억함
    # fit: 학습. 교과서를 5번 반복해서 읽으며 "이런 모양은 1이야, 이런 모양은 7이야" 라고 스스로 규칙을 찾아냄
    # 결과: ai는 28*28 픽셀짜리 작은 흑백 사진을 보면 그게 무슨 숫자인지 맞출 수 있는 능력을 갖게 됨
    model.fit(train_images, train_labels, epochs=5, verbose=0)
    return model

model = train_digital_model()

password = [1,2,3,4]
input_digits = []
status_message = "Capture 4 digits (Press 'C')"

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캡을 열 수 없습니다")
    exit()

while True:
    ret, frame = cap.read() # ret은 true와 false를 담음
    if not ret:
        print("프레임을 가져올 수 없습니다")
        break

    flip_frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape # _는 color가 넘어옴
    center_x, center_y = width // 2, height // 2 # 프레임의 가운데가 찍힘

    # ROI 설정 (300*300)
    roi = flip_frame[center_y - 150:center_y + 150, center_x - 150:center_x + 150]
    cv2.rectangle(flip_frame,
                  (center_x - 150, center_y - 150),
                  (center_x + 150, center_y + 150),
                  (0, 0, 255),
                  2)

    cv2.putText(flip_frame, f"Input: {input_digits}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(flip_frame, status_message, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    cv2.imshow("Webcam", flip_frame)

    # 화면 캡처를 위한 키 값 받기
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c' or 'C'):
        # 흑백이어야 하는 이유
        # AI 입장에서는 어디가 바탕이고, 어디가 글씨인가만 알면 된다
        # 색상 정보가 섞여있으면 AI는 이건 빨간색이라 3인가? 파란색이라 8인가? 라고 헷갈려 할 수 있다.
        # 그래서 색을 다 빼버리고(grayScale) 더 나아가 바탕은 검정, 글씨는 하양(이진화)로 극명하게 대비를 시켜주는 것이다
        gray_image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) # 이진화 하기 위해
        gray_image = np.flip(gray_image, 1)
        cv2.imwrite("gray_image.png", gray_image)

        # 전처리를 꼼꼼하게 했다?
        # 카메라로 숫자를 찍으면 조명 때문에 어둡게 나오기도 하고, 종이에 잡티가 있을 수도 있다. 이런 걸 그대로 주면 AI는 "잡티도 숫자의 일부인가?" 라고 착각한다.
        # 노이즈 제거(Blur): 지저분한 점들을 지워서 깨끗하게 만든다
        # 임계값 처리(Threshold): 어설픈 회색을 다 날리고 확실한 검정과 흰색으로 바꾼다
        # 침식/팽창(Erosion/Dilation): 사용자가 펜으로 숫자를 너무 얇게 썼다면, AI가 공부한 '통통한 숫자'들과 비슷해지도록 글씨 두께를 인위적으로 조절한다
        gaussian_blur = cv2.GaussianBlur(gray_image, (5, 5), 3) # 노이즈 제거
        _, otsu_thread = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)# 이진화
        cv2.imshow("otsu_thread", otsu_thread) # 딱 숫자만 나오게 해야함 (손이 나올수도 있고.. 글자를 진하게 만들어야함), 영상 축소?, 전처리 해줘야함

        ### morph
        kernal = np.ones((5, 5), np.uint8)
        erosion = cv2.erode(otsu_thread, kernal, iterations=5) # 글자가 두꺼워짐 - 배경이 흰색, 물체가 검은색이므로 침식을 해야함. 따라서 검은색이 퍼져나감
        cv2.imshow("erosion", erosion)
        cv2.imwrite("digit_binary_image.png", erosion)

        img = cv2.imread("digit_binary_image.png", cv2.IMREAD_UNCHANGED)
        h, w = img.shape[:2]
        crop_size = 280 # 300 바이 300 에서 20 픽셀정도를 날려버림
        cx, cy = w // 2, h // 2 # 센터
        half = crop_size // 2
        x1, x2 = cx - half, cx + half
        y1, y2 = cy - half, cy + half

        # 경계면 설정
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)

        cropped_img = img[y1:y2, x1:x2]
        cv2.imshow("cropped_image", cropped_img)

        # 이미지 반전
        reversed_img = cv2.bitwise_not(cropped_img)
        cv2.imshow("reversed_image", reversed_img)
        cv2.imwrite("IMAGE_FOR_TEST.png", reversed_img)

        resized_img = cv2.resize(cropped_img, (28, 28), interpolation=cv2.INTER_AREA)
        normalized_img = resized_img / 255.0
        input_data = normalized_img.reshape(1, 28, 28)

        # 예측(predict): 이 사진은 뭐야? 라고 물어보면 ai는 0부터 9까지 각각의 숫자일 확률을 내놓음 (3일 확률 95%, 8일 확률 2%)
        prediction = model.predict(input_data)
        print(f"input_data : {input_data}")

        # 가장 높은 값 뽑기(argmax): 그 중 확률이 가장 높은 숫자 하나를 최종 결과로 선택
        digit = np.argmax(prediction)
        print(f"digit : {digit}")

        # 숫자 저장: c 키를 누를 때마다 AI가 읽은 숫자를 하나씩 리스트에 차곡차곡 담는다
        input_digits.append(int(digit))
        print(f"input_digits : {input_digits}")
        status_message = f"Captured: {digit}"

        # 비밀번호 대조: 숫자가 4개가 모이면 내가 설정한 비밀번호[1,2,3,4]와 똑같은지 비교한다
        if len(input_digits) == 4:
            # 맞으면 성공, 틀리면 실패 메시지를 띄운다
            if input_digits == password:
                status_message = "SUCCESS! Welcome!"
            else:
                status_message = "WRONG PASSWORD! Resetting..."

            print(f"Final Input: {input_digits}, Result: {status_message}")
            input_digits = []

        # 이미지 축소를 해주어야한다 (학습된 이미지는 28*28)
        # 네번 캡쳐해서 학습된 모델 가지고 4번 보여주거나 이미지 저장된걸 보여줘가지고 비번 맞추기?
        # 손으로 터치를 하던가 해야하는데 그럴 수 없어서 이미지로 하는것?
        # 키오스크로 하면 될듯
        # 4개 이미지 입력받아서 뭔가 실행되는걸 하는것
        # 동영상과 찍는것 하기
        # 마치고 찰칵, 마치고 찰칵 -> 어서오세요! 같은 액션
        # 여기에 이미지 프로세싱이랑 머신러닝 들어감
        # 학습을 위해 이미지 조작해보고, dataset 조작
        # 어떻게 필터를 구현했고 (내가 어떻게 필터를 구현했고)
        # 내가 히든레이어 얼마나 썼고, 정확도 몇퍼센트고, OpenCV 라이브러리 사용해서 숫자가 얇으니까 두껍게 전처리해서 필터링까지 해서 던졌다 이런걸 PPT에 나타내기
    elif key == 27:
        break
    #
    # if cv2.waitKey(1) == 27:
    #     break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'tensorflow'